<a href="https://colab.research.google.com/github/robotics-upo/rva-course-material/blob/master/imageprocessingbasics/filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Filtering in OpenCV

In this lab session we will use the following tools:

*   **OpenCV**: http://opencv.org
*   **Numpy**, for handling multidimensional arrays (like images): https://numpy.org/
*   **Matplotlib**, library for visualization in python: https://matplotlib.org/

We will use the 3.x version of OpenCV’s API. We will intensively refer to the documentation (https://opencv-python-tutroals.readthedocs.io/en/latest/index.html)

We analyze the linear filtering operations in OpenCV and use them to extract features of interest like borders and points



In [ ]:
#OpenCV module
import cv2

#Numpy module
import numpy as np

#We can use OpenCV in Colab, but not its functions for creating plots
#We use matplotlib for generating plots
from matplotlib import pyplot as plt

#We use the library scikit to read images from url 
#In OpenCV, the function to read from file is cv2.imread
from skimage import io


As seen in the slides, one very common region-based operation is filtering. 

Linear filters operate by convolving a **kernel** with the image.

* 2D linear filters, convolutions: 
  * https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

We will be using some simple kernels to extract borders in the image.


In [ ]:
im = io.imread('https://robotics.upo.es/~lmercab/rva/background.jpg')

plt.figure()  
plt.imshow(im)

#Kernel for edge detection
filter_kernel = np.array([[-1,-2,-1],
				[0,0,0],
				[1,2,1]])

print(filter_kernel)

#Apply filter over the blue channel. -1 indicates that the pixel type in the output is the same
#as the input (in this case, uint8)
h_edges = cv2.filter2D(im[:,:,0],-1,filter_kernel)

#Type of pixels
print(h_edges.dtype)

plt.figure()  
plt.imshow(h_edges,cmap=plt.cm.gray)
plt.title('Filtered')



Now we have to be careful with the types of the images. Filtering operations will lead to outputs that typically are real numbers, while the pixels in the original image in this case are coded as `uint8`

In [ ]:
#The problem with the former case is that, as the output is uint8, negative numbers
#are truncated to 0. That is edges from bright to dark do not appear
#Alternatively, we can store the output as floats	
h_edges_2 = cv2.filter2D(im[:,:,0],cv2.CV_64F,filter_kernel)

#Type of pixels
print(h_edges_2.dtype)

#Get now the absolute value
abs_h_edges64f = np.absolute(h_edges_2)

#Check the difference
plt.figure()  
plt.imshow(abs_h_edges64f,cmap=plt.cm.gray)
plt.title('Filtered')

The kernels are also numpy arrays.

By transposing the former kernel, we obtain a filter that "responds" vertical edges

In [ ]:
#Same as above
v_edges_2 = cv2.filter2D(im[:,:,0],cv2.CV_64F,np.transpose(filter_kernel))
abs_v_edges64f = np.absolute(v_edges_2)

plt.figure()  
plt.imshow(abs_v_edges64f,cmap=plt.cm.gray)
plt.title('Vertical edges kernel')


# Homework #1

Play with the thresholds and add binary operations to obtain a clean binary edge image

In [ ]:
#We can add the outputs to obtain an image
#with the response of both filters (in this case, the absolute value)
abs_edges_64f = abs_v_edges64f + abs_h_edges64f

plt.figure()  
plt.imshow(abs_edges_64f,cmap=plt.cm.gray)
plt.title('Edge response')

#Check the values of abs_edges_f64. They not need to be between 0 and 255
print('Maximum', np.max(abs_edges_64f))
print('Minimum', np.min(abs_edges_64f))


#We can apply the threshold operation to extract the stongest responses
ret,binary_edges = cv2.threshold(abs_edges_64f,100,255,cv2.THRESH_BINARY)

plt.figure()  
plt.imshow(binary_edges,cmap=plt.cm.gray)
plt.title('Binarized edges ')

print(binary_edges.dtype)
print('Maximum', np.max(binary_edges))
print('Minimum', np.min(binary_edges))


# Smoothing filters

One filter that is used typically to remove noise is an smoothing filter.

An smoothing filter averages the region surrounding the pixel of interest.

If we want to give more importance to zones closer to the pixel, typicall a Gaussian kernel is used (the Gaussian kernel has many nice properties as well, but we will not see them now)

In [ ]:
#Create a Gaussian kernel
#Size
k=10
#Sigma
sigma=5

gaussKernel=np.zeros(shape=(k,k))

#Apply the Gaussian 'bell' equation
for i in range(k):
  for j in range(k):
    gaussKernel[i,j] = np.exp(-((i-(k-1)/2)**2+(j-(k-1)/2)**2)/sigma**2)

#Normalize the kernel (so all element sum 1.0)
gaussKernel /= np.sum(gaussKernel)

plt.figure()  
plt.imshow(gaussKernel,cmap=plt.cm.gray)
plt.title('Gaussian Kernel')

Applying the Gaussian kernel to an image gives the same image smoothed.

**TODO**: Play with the values of *sigma* and the kernel size *k* to see the effect on the image

In [ ]:
#Apply to the original image
#If we use a colour image, the kernel is applied to each channel
#independently
blurred_image = cv2.filter2D(im,-1,gaussKernel)

plt.figure(figsize=(14,14))
plt.subplot(1,2,1)
plt.imshow(im)
plt.title('Original')
plt.subplot(1,2,2)
plt.imshow(blurred_image)
plt.title('Blurred image')

# Shaperning


In [ ]:
#Sharpening filter

#Average filter: returns the average on a 3x3 neighbourhood
average_kernel = np.ones((3,3),np.float32)/9.0

#Delta kernel: returns (twice) the value of the pixel
delta_kernel = np.array([[0,0,0],
				[0,1,0],
				[0,0,0]],np.float32)

	
print(average_kernel)
print(delta_kernel)

#The sharpening kernel is a combination of the other two
#We suubtract the average to twice the value at a pixel
sharpening_kernel = 2*delta_kernel - average_kernel

print(sharpening_kernel)

#If we use a colour image, the kernel is applied to each channel
#independently
sharpened_image = cv2.filter2D(im,-1,sharpening_kernel)

plt.figure(figsize=(14,14))
plt.imshow(im)
plt.title('Original')

plt.figure(figsize=(14,14))
plt.imshow(sharpened_image)
plt.title('Sharpened image')
